In [133]:
#----------Cell 1----------


from groq import Groq
from dotenv import load_dotenv
import os
import pprint
import json
import re

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [134]:
test_input = "What the    fjfj fuck"

In [135]:
inp = test_input.split()

In [136]:
_.join(inp)

'What../model_output_data/task_json_data\\research_on_javathe../model_output_data/task_json_data\\research_on_javafjfj../model_output_data/task_json_data\\research_on_javafuck'

In [137]:
#----------Cell 2----------


user_input = "Research On Java"
research_context = "Starting fresh research"

In [138]:
#----------Paths----------



task_data_path = "../model_output_data/task_json_data"

#-------Cleaning Input For Folder Name-------

lower_user_input = user_input.lower()

# Matches any character NOT in the range a-z, A-Z, or 0-9
cleaned_text = re.sub(r'[^a-zA-Z0-9\s]+', '', lower_user_input)

folder_name_for_query = re.sub(r"\s+", "_", cleaned_text)

complete_data_path_query = os.path.join(
    task_data_path,
    folder_name_for_query
)

if not os.path.exists(complete_data_path_query):
    os.makedirs(complete_data_path_query, exist_ok=True)
    print(f"Created folder: {complete_data_path_query}")
else:
    print(f"Folder already exists: {complete_data_path_query}")


Folder already exists: ../model_output_data/task_json_data\research_on_java


In [139]:
complete_data_path_query

'../model_output_data/task_json_data\\research_on_java'

In [140]:
#----------Cell 3----------


messages = []
# MAX_MESSAGES = 10   # total, user + assistant



# if len(messages) > MAX_MESSAGES:
#     messages = messages[-MAX_MESSAGES:]




In [141]:
#----------Cell 4----------

system_prompt = f"""
You are creating an initial research plan for the topic: "{user_input}"
Initial Query: "{user_input}"
Research Context: {research_context if research_context else "Starting fresh research"}
Decompose this query into 3–5 actionable research tasks. Return a JSON array with each task having:
• "description": Clear, actionable task (string)
• "priority": 1–10 (integer, higher = more important, default=5)
• "type": "research" (always "research")
Focus on: understanding the topic, gathering information, identifying key aspects, and building foundational knowledge.
Example for "Impacts of Generative AI on Scientific Research":
<answer>
[
{{"description": "Survey major applications of generative AI in scientific discovery", "priority": 8, "type": "research"}},
{{"description": "Identify key papers and institutions leading AI-assisted science research", "priority": 7, "type": "research"}},
{{"description": "Examine methodological advances enabled by generative models in ...", "priority": 6, "type": "research"}},
{{"description": "Assess challenges and ethical considerations of AI-generated scientific results", "priority": 5, "type": "research"}}
]
</answer>
CRITICAL: Wrap JSON in <answer>tags.
Output ONLY valid JSON.


"""

messages.append({"role": "system", "content": system_prompt})

In [142]:
#----------Cell 5----------


messages.append({"role": "user", "content": user_input}) 

task_agent = Groq()
completion = task_agent.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=messages,
    stream=False
)

reply = completion.choices[0].message.content
print(reply)
messages.append({"role": "assistant", "content": reply})

<answer>
[
{"description": "Investigate the history and evolution of the Java programming language", "priority": 5, "type": "research"},
{"description": "Gather information on the design goals, features, and philosophy of Java", "priority": 7, "type": "research"},
{"description": "Identify key aspects of Java, including memory management, security, and platform independence", "priority": 8, "type": "research"},
{"description": "Research popular use cases, industries, and applications that utilize Java", "priority": 6, "type": "research"},
{"description": "Explore the most popular libraries, frameworks, and tools used in Java development", "priority": 8, "type": "research"}
]
</answer>


In [143]:
print(messages[-1]['content'])

<answer>
[
{"description": "Investigate the history and evolution of the Java programming language", "priority": 5, "type": "research"},
{"description": "Gather information on the design goals, features, and philosophy of Java", "priority": 7, "type": "research"},
{"description": "Identify key aspects of Java, including memory management, security, and platform independence", "priority": 8, "type": "research"},
{"description": "Research popular use cases, industries, and applications that utilize Java", "priority": 6, "type": "research"},
{"description": "Explore the most popular libraries, frameworks, and tools used in Java development", "priority": 8, "type": "research"}
]
</answer>


In [144]:
assistant_output = messages[-1]["content"]

In [145]:
start = assistant_output.find("<answer>") + len("<answer>")
end = assistant_output.find("</answer>")

json_text = assistant_output[start:end].strip()

tasks = json.loads(json_text)

with open(os.path.join(complete_data_path_query, "tasks.json"), "w") as f:
    json.dump(tasks, f, indent=4)

print(tasks)

[{'description': 'Investigate the history and evolution of the Java programming language', 'priority': 5, 'type': 'research'}, {'description': 'Gather information on the design goals, features, and philosophy of Java', 'priority': 7, 'type': 'research'}, {'description': 'Identify key aspects of Java, including memory management, security, and platform independence', 'priority': 8, 'type': 'research'}, {'description': 'Research popular use cases, industries, and applications that utilize Java', 'priority': 6, 'type': 'research'}, {'description': 'Explore the most popular libraries, frameworks, and tools used in Java development', 'priority': 8, 'type': 'research'}]
